# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida, de llegada y duración
- Información de las escalas

¡Empecemos!
la pagina para mexico es <a href="https://www.latamairlines.com/mx/es">latamairlines mx</a>

debes entender que no puedes buscar vuelos a cualquier parte del mundo, ya que hay rutas establecidas, para encontrar una ruta turistica ve a la seccion de "ofertas de vuelo desde CDMX", asi encontre que rio es un pais muy visitado. 
el link que usaremos es para buscar vuelos con las siguientes caracteristicas:
- viaje solo de ida, economico, adulto, desde CDMX a rio de janeiro
- fecha 15 dias de la fecha actual osea para el 15 de febrero   
este es el link generado:   
https://www.latamairlines.com/mx/es/ofertas-vuelos?origin=MEX&inbound=null&outbound=2023-02-15T18%3A00%3A00.000Z&destination=RIO&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED_A


## inteto del objetivo usando requests y beautiful soup
intentaremos resolver la tarea usando lo aprendido en el anterior notebook 

In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://www.latamairlines.com/mx/es/ofertas-vuelos?origin=MEX&inbound=null&outbound=2023-02-15T18%3A00%3A00.000Z&destination=RIO&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED_A"
r = requests.get(url)
r.status_code

200

In [3]:
s = BeautifulSoup(r.text, 'lxml')
#print(s.prettify())

En la respuesta aparecera que habilitemos JS para ver todo el texto, esto es debido a que python no sabe interpretar JS. la respuesta no contiene la información que buscamos, ya que la misma aparece recién después de ejecutar el código JavaSCript que está en la respuesta.

## Selenium
Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.
- Paso 1: instanciar un **driver** del navegador
- Paso 2: hacer que el navegador cargue la página web.
- Paso 3: extraer la información de la página
- Paso 4: cerrar el navegador

In [4]:
from selenium import webdriver

In [19]:
# definimos opciones para nuestro navedor que controlaremos desde selenium python
options = webdriver.FirefoxOptions()
options.add_argument("-private") # indicamos que sea una ventana incognita

#usamos el driver para controlar una pestaña nueva de chrome desde el codigo de python mediante selenium
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe', options=options)
driver.get(url) # en la pestaña nuva abrimos la pagina deseada 

C:\Users\Panda\AppData\Local\Temp\ipykernel_10980\329548938.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe', options=options)


SessionNotCreatedException: Message: session not created: No matching capabilities found
Stacktrace:
Backtrace:
	(No symbol) [0x00266643]
	(No symbol) [0x001FBE21]
	(No symbol) [0x000FDA9D]
	(No symbol) [0x00155D69]
	(No symbol) [0x001553A0]
	(No symbol) [0x0015670C]
	(No symbol) [0x0015655C]
	(No symbol) [0x0014FB76]
	(No symbol) [0x001249C1]
	(No symbol) [0x00125E5D]
	GetHandleVerifier [0x004DA142+2497106]
	GetHandleVerifier [0x005085D3+2686691]
	GetHandleVerifier [0x0050BB9C+2700460]
	GetHandleVerifier [0x00313B10+635936]
	(No symbol) [0x00204A1F]
	(No symbol) [0x0020A418]
	(No symbol) [0x0020A505]
	(No symbol) [0x0021508B]
	BaseThreadInitThunk [0x76BA00F9+25]
	RtlGetAppContainerNamedObjectPath [0x770A7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x770A7B8E+238]
	(No symbol) [0x00000000]


In [18]:
driver.close() # cerramos el navegador de enlace 